In [32]:
# Directories
import os

new_directory = r'c://Users//Fer//TESIS_ARCHIVOS//TESIS_AIRE//MP_Forecasting//mp_forecasting//notebooks'
os.chdir(new_directory)

# Data Manipulation
import pandas as pd # for data manipulation
import numpy as np # for data manipulation

# Guardar resultados
import pickle

# Utils

from training_code.utils import utils_xgboost

# Tiempo
import datetime as dt
from dateutil.relativedelta import relativedelta, MO

# Metricas
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.multioutput import MultiOutputRegressor


# Visualizations
import plotly.graph_objects as go # for data visualization
import plotly.express as px # for data visualization
import matplotlib.pyplot as plt

# Advertencias
import warnings
warnings.filterwarnings("ignore")

In [8]:
datos = pd.read_csv('datos/230127_train_ESTACIONES.csv', parse_dates=['FECHAHORA'])
validacion = pd.read_csv('datos/230127_test_ESTACIONES.csv', parse_dates=['FECHAHORA'])

In [9]:
datos.head()

,ESTACION,FECHAHORA,ANHO,DIA,MES,HORA,MINUTO,MP1,MP2_5,MP10,...,DIA_SEM,AQI_MP2_5,AQI_MP10,MEDICION_DIA,MP1_ANTERIOR,MP2_5_ANTERIOR,MP10_ANTERIOR,TEMPERATURA_PRONOSTICO,HUMEDAD_PRONOSTICO,PRESION_PRONOSTICO
0,1,2019-12-30 13:00:00,2019,30,12,13,0,2.839,3.989,4.599,...,0,72.0,25.0,157,2.609,3.720,4.519,36.5,42.8,995.1
1,1,2019-12-30 13:05:00,2019,30,12,13,5,9.000,13.079,21.319,...,0,72.0,25.0,158,1.919,2.659,10.340,36.7,42.1,994.9
2,1,2019-12-30 13:10:00,2019,30,12,13,10,1.969,2.369,13.170,...,0,72.0,25.0,159,1.769,2.609,3.419,36.6,43.5,994.8
3,1,2019-12-30 13:15:00,2019,30,12,13,15,1.740,2.379,2.429,...,0,72.0,25.0,160,1.480,2.009,2.159,37.0,41.4,994.7
4,1,2019-12-30 13:20:00,2019,30,12,13,20,2.710,4.119,7.710,...,0,72.0,25.0,161,1.250,1.909,2.809,37.1,40.6,994.6


In [12]:
data = datos[datos['ESTACION'] == 1]

train = data[['FECHAHORA','AQI_MP2_5']]

train.index = train['FECHAHORA']

del train['FECHAHORA']

data = validacion[validacion['ESTACION'] == 1]

test = data[['FECHAHORA','AQI_MP2_5']]

test.index = test['FECHAHORA']

del test['FECHAHORA']

In [39]:
from pmdarima.arima import auto_arima

estacion = 1

variables = ["ANHO", 'DIA', 'MES', 'HORA', 'MINUTO', 'MP1', 'MP2_5', 'MP10', 
                'TEMPERATURA', 'HUMEDAD', 'PRESION', 'TEMPERATURA_PRONOSTICO', 
                'HUMEDAD_PRONOSTICO', 'PRESION_PRONOSTICO', 'DIA_SEM', 'TRAFICO' , 'AQI_MP10', 'AQI_MP2_5']

dependent = ['AQI_MP2_5']

number_of_features = len(variables)

training_days = 2 
forecast_days = 1 
samples_per_day = 288
step = 288/4

# Creamos una variable que nos diga con cuantos meses de entrenamiento queremos contar para el X_train
train_months = relativedelta(months = 12)
test_months = relativedelta(months = 2, days = 20)

input_samples = int(samples_per_day * training_days) # cantidad de muestras en 7 dias
output_samples = int(samples_per_day * forecast_days) # cantidad de muestras en 1 dia
train_test_samples = int(input_samples + output_samples) # cantidad de datos para el train_test



X_train, y_train, X_test, y_test = utils_xgboost.get_everything(datos, 
                                                                estacion,
                                                                train_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)

X_train_val, y_train_val, X_test_val, y_test_val = utils_xgboost.get_everything(validacion, 
                                                                estacion,
                                                                test_months, 
                                                                variables, 
                                                                dependent, 
                                                                train_test_samples, 
                                                                input_samples, 
                                                                output_samples, 
                                                                number_of_features,
                                                                step)

y_train = y_train.flatten()
y_test = y_test.flatten()

model = auto_arima(y_train, trace=True, error_action='ignore', suppress_warnings=True)

trained_arima = MultiOutputRegressor(model.fit(y_train))
# forecast = trained_arima.predict(n_periods=len(y_test))
# forecast = pd.DataFrame(forecast,index = test.index,columns=['Prediction'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=1712259.849, Time=39.09 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1712266.795, Time=6.26 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=1712260.658, Time=13.35 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=1712260.724, Time=15.51 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=1712264.795, Time=3.19 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=1712257.873, Time=44.33 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=1712255.857, Time=20.03 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=1712255.666, Time=32.62 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=1712257.691, Time=54.05 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=1712242.362, Time=41.02 sec
 ARIMA(1,1,4)(0,0,0)[0] intercept   : AIC=1712244.435, Time=64.87 sec
 ARIMA(0,1,5)(0,0,0)[0] intercept   : AIC=1712226.462, Time=65.33 sec
 ARIMA(1,1,5)(0,0,0)[0] intercept   : AIC=1712099.710, Time=188.45 sec
 ARIMA(2,1,5)(0,0,0)[0] intercept   : AIC=171209

In [36]:
trained_arima = MultiOutputRegressor(model.fit(y_train))


In [37]:
forecast = trained_arima.predict(y_train)
forecast = pd.DataFrame(forecast,index = test.index,columns=['Prediction'])

NotFittedError: This MultiOutputRegressor instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [18]:
# mape = mean_absolute_percentage_error(np.asarray(forecast.Prediction), np.asarray(test.AQI_MP2_5))
# mae = mean_absolute_error(np.asarray(forecast.Prediction), np.asarray(test.AQI_MP2_5))

mape = mean_absolute_percentage_error(forecast, y_test)
mae = mean_absolute_error(forecast.Prediction, y_test)

print(mape)
print(mae)

0.40620779366413734
21.527422575503635


In [23]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(y = list(forecast.Prediction), name= 'forecast')
    )
fig.add_trace(
    go.Scatter(y = list(test.AQI_MP2_5), name = 'data')
)

fig.update_layout( title_text = "Forecasts vs Targets")